In [39]:
from bs4 import BeautifulSoup
import urllib
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import feather
import requests



#function to get the links from https://www.reuters.com/resources/archive/us/DATE.html
#DATE is in YYYYMMDD format
#get two dates from user:start date and end date. Scrape all articles between these dates 
def get_links():
    #get input
    start_date = datetime.strptime(input("Please enter the start date:"), '%Y%m%d')
    end_date=datetime.strptime(input("Please enter the end date:"), '%Y%m%d')
    #create list of dates between start and end dates
    delta=end_date-start_date
    date_list=[start_date + timedelta(i) for i in range(delta.days + 1)]
    link_list=[]
    #scrape what the links of articles for each date. put them in a list.
    for i in date_list:
        p = urllib.request.urlopen('https://www.reuters.com/resources/archive/us/'+i.strftime('%Y%m%d')+'.html').read()
        soup = BeautifulSoup(p,'lxml')
        content=soup.find_all("div",class_="module")
        if len(content)>=1:
            for i in content[0].find_all(class_='headlineMed'): 
                link_list.append(i.a['href'])            
    return link_list

#function to get the dates of each article
#use the link list created by the "get_links" function and get timestamps of each article
def get_dates(links):
    date_list=[]
    for i in links:
        try:
            y=urllib.request.urlopen(i).read()  
        except Exception as e:
            date_list.append(e)
            continue
        z = BeautifulSoup(y,'lxml')
        a=z.find_all("div",class_="date_V9eGk")
        #print(len(a))
        if len(a)>=1:
            date_list.append(a[0].get_text()[0:23])
        elif len(a)==0:
            date_list.append("Na")                    
    return date_list

#function to get the keywords of each article
#use the link list created by the "get_links" function and get keywords of each article
def get_keywords(links):
    keyword_list=[]
    for i in links:
        try:
            y=urllib.request.urlopen(i).read()
        except Exception as e:
            keyword_list.append(e)
            continue
        z = BeautifulSoup(y,'lxml')
        a_keys=z.find_all('meta',attrs={'name':'keywords'})
        #print(len(a))
        if len(a_keys)>=1:
            keyword_list.append(a_keys[0]['content'].lower().split(','))
        elif len(a)==0:
            keyword_list.append("Na")                        
    return keyword_list


#function to get the titles 
#use the link list created by the "get_links" function and get titles of each article
def get_titles(links):
    title_list=[]
    for i in links:
        try:
            y=urllib.request.urlopen(i).read()     
        except Exception as e:
            title_list.append(e)
            continue
        z = BeautifulSoup(y,'lxml')
        a_titles=z.find_all("title")
        if len(a_titles)>=1:
            title_list.append(a_titles[0].get_text()#[17:-10])
        elif len(a)==0:
            title_list.append("Na")            
    return title_list

#function to get the articles 
#use the link list created by the "get_links" function and get titles of each article
def get_articles(links):
    article_list=[]
    for i in links:
        try:
            y=urllib.request.urlopen(i).read()
       
        except Exception as e:
            print(e)
            continue
        
        z = BeautifulSoup(y,'lxml')
        a=z.find_all("div",class_="body_1gnLA")
        #print(len(a))
        if len(a)>=1:
            start_time = time.time()
            article_list.append(a[0].get_text())
            print("--- %s seconds ---" % (time.time() - start_time))
        elif len(a)==0:
            article_list.append('Na')
    return article_list
        
#run the functions
#links=get_links()
#keywords=get_keywords(links)
#dates=get_dates(links)
#titles=get_titles(links)

#put everything in a n * 3 dataframe where n is the number of articles and the columns are dates, links and keywords
#respectively. you need to do some cleaning to get to the appropriate format of the dates. 
#df_1 = pd.DataFrame(np.column_stack([dates, links, keywords,titles]),columns=['date', 'link', 'keywords','titles']) 
#df_1['date'] = df_1['date'].astype(str)
#df_1['new']=df_1.loc[:,'date'].str[0:23]
#df_1.drop('date',axis=1,inplace=True)
#df_1.rename(columns={'new':'date'}, inplace=True)

#save the dataframe in feather format which is a binary storage format that can be read by both python and R
#write_feather(df_1, 'MainDF.feather')

In [2]:
#time the get_links function 
import time
start_time = time.time()
links=get_links()
print("--- %s seconds ---" % (time.time() - start_time))

Please enter the start date:20180516
Please enter the end date:20180601
--- 15.812518835067749 seconds ---


In [3]:
#see how many links you have between those dates
len(links)

14452

In [3]:
#time the get_keywords function 
start_time = time.time()
keywords=get_keywords(links)
print("--- %s seconds ---" % (time.time() - start_time))


--- 92.95884704589844 seconds ---


In [11]:
#time the get_articles function
start_time = time.time()
articles=get_articles(links)#420 articles
print("--- %s seconds ---" % (time.time() - start_time))


--- 2.7179718017578125e-05 seconds ---
--- 3.790855407714844e-05 seconds ---
--- 2.7894973754882812e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 0.00010418891906738281 seconds ---
--- 3.24249267578125e-05 seconds ---
--- 4.38690185546875e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 3.600120544433594e-05 seconds ---
--- 4.696846008300781e-05 seconds ---
--- 4.1961669921875e-05 seconds ---
--- 3.719329833984375e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 2.8848648071289062e-05 seconds ---
--- 2.7179718017578125e-05 seconds ---
--- 4.315376281738281e-05 seconds ---
--- 3.719329833984375e-05 seconds ---
--- 2.6702880859375e-05 seconds ---
--- 3.123283386230469e-05 seconds ---
--- 3.719329833984375e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 3.910064697265625e-05 seconds ---
--- 2.6941299438476562e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 2.6226043701171875e-05 seconds ---
--- 2.4795

--- 1.6927719116210938e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 1.5735626220703125e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 1.5735626220703125e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 1.9788742065429688e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 1.5974044799804688e-05 seconds ---
--- 2.2649765014648438e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 2.9087066650390625e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 4.124641418457031e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 2.7894973754882812e-05 seconds ---
--- 3.695487976074219e-05 seconds ---
--- 2.7894973754882812e-05 seconds ---
--- 3.600120544433594e-05 se

--- 2.5987625122070312e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 4.696846008300781e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 2.9325485229492188e-05 seconds ---
--- 4.7206878662109375e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 3.62396240234375e-05 seconds ---
--- 1.8358230590820312e-05 seconds ---
--- 1.9311904907226562e-05 seconds ---
--- 2.9802322387695312e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 2.8848648071289062e-05 seconds ---
--- 4.482269287109375e-05 seconds ---
--- 2.9802322387695312e-05 seconds ---
--- 5.507469177246094e-05 seconds ---
--- 4.410743713378906e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 3.2901763916015625e-05 seconds ---
--- 1.52587890625e-05 seconds ---
--- 6.985664367675781e-05 seconds ---
--- 3.790855407714844e-05 seconds ---
--- 2.574920654296875e-05 seconds ---
--- 1.5

--- 1.811981201171875e-05 seconds ---
--- 1.5974044799804688e-05 seconds ---
--- 2.193450927734375e-05 seconds ---
--- 5.984306335449219e-05 seconds ---
--- 1.811981201171875e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 1.5020370483398438e-05 seconds ---
--- 2.384185791015625e-05 seconds ---
--- 1.9311904907226562e-05 seconds ---
--- 1.9073486328125e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 1.5974044799804688e-05 seconds ---
--- 1.9073486328125e-05 seconds ---
--- 1.52587890625e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 1.6689300537109375e-05 seconds ---
--- 4.887580871582031e-05 seconds ---
--- 1.621246337890625e-05 seconds ---
--- 1.5020370483398438e-05 seconds ---
--- 1.5735626220703125e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 1.5735626220703125e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 3.790855407714844e-05 seconds ---
--- 3.886222839355469e-05 seconds ---
--- 3.623

--- 2.193450927734375e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 5.2928924560546875e-05 seconds ---
--- 2.5987625122070312e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 2.7894973754882812e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 5.2928924560546875e-05 seconds ---
--- 5.2928924560546875e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 2.7179718017578125e-05 seconds ---
--- 1.811981201171875e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 2.7179718017578125e-05 seconds ---
--- 1.621246337890625e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 3.814697265625e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 3.218650817871094e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
---

...

--- 2.5987625122070312e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 3.1948089599609375e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 2.8133392333984375e-05 seconds ---
--- 2.8133392333984375e-05 seconds ---
--- 7.987022399902344e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 2.9802322387695312e-05 seconds ---
--- 4.935264587402344e-05 seconds ---
--- 4.601478576660156e-05 seconds ---
--- 2.7894973754882812e-05 seconds ---
--- 5.602836608886719e-05 seconds ---
--- 5.1021575927734375e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 4.792213439941406e-05 seconds ---
--- 4.38690185546875e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 2.574920654296875e-05 seconds ---
--- 1.9073486328125e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 5.1021575927734375e-05 seconds ---
--- 3.075599670410156e-05 seconds ---
--- 2.288818359375e-05 seconds ---
--- 1.9311904907226562e-05 seconds ---
--- 1.7881

--- 2.6941299438476562e-05 seconds ---
--- 1.5974044799804688e-05 seconds ---
--- 3.981590270996094e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 1.4781951904296875e-05 seconds ---
--- 2.6941299438476562e-05 seconds ---
--- 3.528594970703125e-05 seconds ---
--- 3.886222839355469e-05 seconds ---
--- 1.811981201171875e-05 seconds ---
--- 4.363059997558594e-05 seconds ---
--- 6.29425048828125e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 4.220008850097656e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 2.8133392333984375e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 1.9073486328125e-05 seconds ---
--- 1.811981201171875e-05 seconds ---
--- 1.811981201171875e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 4.482269287109375e-05 seconds ---
--- 2.9087066650390625e-05 seconds ---
--- 4.1961669921875e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 1.

--- 4.315376281738281e-05 seconds ---
--- 2.8848648071289062e-05 seconds ---
--- 3.0040740966796875e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 2.193450927734375e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 5.412101745605469e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 0.00014281272888183594 seconds ---
--- 4.7206878662109375e-05 seconds ---
--- 7.390975952148438e-05 seconds ---
--- 0.00011801719665527344 seconds ---
--- 5.602836608886719e-05 seconds ---
--- 3.2901763916015625e-05 seconds ---
--- 2.6226043701171875e-05 seconds ---
--- 5.888938903808594e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 2.0265579223632812e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 5.602836608886719e-05 seconds ---
--- 5.4836273193359375e-05 seconds ---
--- 0.00011587142944335938 seconds ---
--- 5.1021575927734375e-05 seconds ---
--- 5.412101745605469e-05 seconds ---
--- 4.291534423828125e-05 seconds ---
--- 9.703636169433594e-05 seconds ---

--- 3.695487976074219e-05 seconds ---
--- 4.100799560546875e-05 seconds ---
--- 4.315376281738281e-05 seconds ---
--- 1.9073486328125e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 1.5974044799804688e-05 seconds ---
--- 1.5735626220703125e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 6.699562072753906e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 4.482269287109375e-05 seconds ---
--- 2.9802322387695312e-05 seconds ---
--- 2.9325485229492188e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 3.695487976074219e-05 seconds ---
--- 2.193450927734375e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 5.1021575927734375e-05 seconds ---
--- 1.8358230590820312e-05 seconds ---
--- 4.410743713378906e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 3.0040740966796875e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 2.47955322265625e-05 seconds ---
--- 

--- 1.7881393432617188e-05 seconds ---
--- 6.389617919921875e-05 seconds ---
--- 5.698204040527344e-05 seconds ---
--- 4.100799560546875e-05 seconds ---
--- 4.00543212890625e-05 seconds ---
--- 2.6226043701171875e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 4.696846008300781e-05 seconds ---
--- 1.5735626220703125e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 1.5020370483398438e-05 seconds ---
--- 2.384185791015625e-05 seconds ---
--- 2.6226043701171875e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 0.00013637542724609375 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 9.894371032714844e-05 seconds ---
--- 4.00543212890625e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 3.814697265625e-05 seconds ---
--- 2.193450927734375e-05 seconds ---
--- 3.886222839355469e-05 seconds ---
--- 4.982948303222656e-05 seconds ---
--- 1.9073486328125e-05 seconds ---
--- 1.59

In [12]:
len(articles)


14452

In [17]:

import pickle

pickle.dump( articles, open( "articles20180516-20180601.p", "wb" ) )
... 

Ellipsis

In [14]:
import os 
os.getcwd()

'/Users/berkayadlim/Desktop'

In [18]:
#time the get_keywords function 
start_time = time.time()
keywords=get_keywords(links)
print("--- %s seconds ---" % (time.time() - start_time))


--- 3212.4002029895782 seconds ---


In [19]:
#time the get_keywords function 
start_time = time.time()
keywords=get_keywords(links)
print("--- %s seconds ---" % (time.time() - start_time))


--- 2736.5334470272064 seconds ---


In [20]:
len(keywords)

14452

In [21]:
len(articles)

14452

In [22]:
len(links)

14452

In [40]:
start_time = time.time()
dates=get_dates(links)
print("--- %s seconds ---" % (time.time() - start_time))

--- 4119.7476642131805 seconds ---


In [24]:
if 'Na' in articles:
    print('YES')

YES


In [41]:
len(dates)

14452

In [43]:
import pickle 
with open('dates201805160601.pkl', 'wb') as f:
    pickle.dump(dates, f)


In [44]:
with open('links201805160601.pkl', 'wb') as f:
    pickle.dump(links, f)


In [45]:
with open('articles201805160601.pkl', 'wb') as f:
    pickle.dump(articles, f)

with open('keywords201805160601.pkl', 'wb') as f:
    pickle.dump(keywords, f)


In [46]:
start_time = time.time()
links_2=get_links()
print("--- %s seconds ---" % (time.time() - start_time))

Please enter the start date:20180601
Please enter the end date:20180621
--- 19.177860021591187 seconds ---


In [49]:
start_time = time.time()
articles_2=get_articles(links_2)
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.288818359375e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 2.2172927856445312e-05 seconds ---
--- 2.6941299438476562e-05 seconds ---
--- 4.1961669921875e-05 seconds ---
--- 3.910064697265625e-05 seconds ---
--- 2.7179718017578125e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 6.079673767089844e-05 seconds ---
--- 3.600120544433594e-05 seconds ---
--- 2.288818359375e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 6.794929504394531e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 3.719329833984375e-05 seconds ---
--- 4.601478576660156e-05 seconds ---
--- 1.9073486328125e-05 seconds ---
--- 2.6226043701171875e-05 seconds ---
--- 3.3855438232421875e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 3.266334533691406e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 3.790855407714844e-05 seconds ---
--- 3.67164611816

--- 3.314018249511719e-05 seconds ---
--- 5.507469177246094e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 2.288818359375e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 2.5987625122070312e-05 seconds ---
--- 2.9087066650390625e-05 seconds ---
--- 2.574920654296875e-05 seconds ---
--- 3.0279159545898438e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 2.7894973754882812e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 1.811981201171875e-05 seconds ---
--- 4.7206878662109375e-05 seconds ---
--- 3.4809112548828125e-05 seconds ---
--- 1.8358230590820312e-05 seconds ---
--- 2.6941299438476562e-05 seconds ---
--- 2.5987625122070312e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 3.695487976074219e-05 seconds ---
--- 3.6716461181640625e-05 seconds ---
--- 5.2928924560546875e-05 seconds ---
--- 3.0994415283203125e-05 seconds -

--- 1.6927719116210938e-05 seconds ---
--- 3.695487976074219e-05 seconds ---
--- 2.2172927856445312e-05 seconds ---
--- 1.6689300537109375e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 4.38690185546875e-05 seconds ---
--- 3.528594970703125e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 3.1948089599609375e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 4.6253204345703125e-05 seconds ---
--- 2.6226043701171875e-05 seconds ---
--- 3.0279159545898438e-05 seconds ---
--- 3.600120544433594e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 1.9311904907226562e-05 seconds ---
--- 3.719329833984375e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 4.506111145019531e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
---

--- 3.1948089599609375e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 3.814697265625e-05 seconds ---
--- 2.8848648071289062e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 2.288818359375e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 4.673004150390625e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 3.1948089599609375e-05 seconds ---
--- 3.0040740966796875e-05 seconds ---
--- 1.811981201171875e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 5.2928924560546875e-05 seconds ---
--- 2.7179718017578125e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 2.6702880859375e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 2.384185791015625e-05 seconds ---
--- 5.507469177246094e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 2.8848648071289062e-05 seconds ---
--- 5.0067901611328125e-05 seconds ---
--- 4.9114227294921875e-05 seconds ---
--- 5.3882598876953125e-05 seconds ---
--- 

--- 1.7881393432617188e-05 seconds ---
--- 1.811981201171875e-05 seconds ---
--- 1.5974044799804688e-05 seconds ---
--- 6.175041198730469e-05 seconds ---
--- 2.8133392333984375e-05 seconds ---
--- 5.698204040527344e-05 seconds ---
--- 3.1948089599609375e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 2.6941299438476562e-05 seconds ---
--- 2.6702880859375e-05 seconds ---
--- 3.0994415283203125e-05 seconds ---
--- 1.9073486328125e-05 seconds ---
--- 4.506111145019531e-05 seconds ---
--- 5.91278076171875e-05 seconds ---
--- 2.8133392333984375e-05 seconds ---
--- 3.814697265625e-05 seconds ---
--- 3.719329833984375e-05 seconds ---
--- 4.1961669921875e-05 seconds ---
--- 1.5974044799804688e-05 seconds ---
--- 1.811981201171875e-05 seconds ---
--- 2.5987625122070312e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 3.314018

...

--- 3.719329833984375e-05 seconds ---
--- 4.8160552978515625e-05 seconds ---
--- 3.4809112548828125e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--- 3.886222839355469e-05 seconds ---
--- 3.814697265625e-05 seconds ---
--- 2.2172927856445312e-05 seconds ---
--- 2.5987625122070312e-05 seconds ---
--- 4.506111145019531e-05 seconds ---
--- 5.2928924560546875e-05 seconds ---
--- 4.38690185546875e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 2.6941299438476562e-05 seconds ---
--- 3.1948089599609375e-05 seconds ---
--- 2.6226043701171875e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 2.9325485229492188e-05 seconds ---
--- 2.288818359375e-05 seconds ---
--- 2.7179718017578125e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 5.2928924560546875e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 2.7179718017578125e-05 seconds ---
--- 5.3882598876953125e-05 seconds ---
--- 5.698204040527344e-05 seconds ---
---

--- 4.792213439941406e-05 seconds ---
--- 4.100799560546875e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 3.886222839355469e-05 seconds ---
--- 4.887580871582031e-05 seconds ---
--- 2.8848648071289062e-05 seconds ---
--- 2.9087066650390625e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 3.695487976074219e-05 seconds ---
--- 3.266334533691406e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 2.288818359375e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 3.0040740966796875e-05 seconds ---
--- 2.09808349609375e-05 seconds ---
--- 2.7894973754882812e-05 seconds ---
--- 3.5762786865234375e-05 seconds ---
--- 2.7179718017578125e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 1.7881393432617188e-05 seconds ---
--- 3.790855407714844e-05 seconds ---
--- 4.410743713378906e-05 seconds ---
--- 1.621246337890625e-05 seconds ---
--- 4.00543212890625e-05 seconds ---
--- 4.100799560546875e-05 seconds ---
--- 4.79

--- 6.29425048828125e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 1.621246337890625e-05 seconds ---
--- 2.0265579223632812e-05 seconds ---
--- 2.288818359375e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 4.00543212890625e-05 seconds ---
--- 2.47955322265625e-05 seconds ---
--- 2.193450927734375e-05 seconds ---
--- 3.886222839355469e-05 seconds ---
--- 5.698204040527344e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 2.8848648071289062e-05 seconds ---
--- 4.7206878662109375e-05 seconds ---
--- 5.3882598876953125e-05 seconds ---
--- 2.9087066650390625e-05 seconds ---
--- 5.030632019042969e-05 seconds ---
--- 3.266334533691406e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 3.790855407714844e-05 seconds ---
--- 2.3126602172851562e-05 seconds ---
--- 3.075599670410156e-05 seconds ---
--- 3.0040740966796875e-05 seconds ---
--- 2.5987625122070312e-05 seconds ---
--- 3.4809112548828125e-05 seconds ---
--- 2.67

--- 3.910064697265625e-05 seconds ---
--- 4.076957702636719e-05 seconds ---
--- 1.5735626220703125e-05 seconds ---
--- 5.412101745605469e-05 seconds ---
--- 3.981590270996094e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 2.6226043701171875e-05 seconds ---
--- 3.981590270996094e-05 seconds ---
--- 3.1948089599609375e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 1.52587890625e-05 seconds ---
--- 2.2649765014648438e-05 seconds ---
--- 2.5033950805664062e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 3.1948089599609375e-05 seconds ---
--- 2.6226043701171875e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 3.910064697265625e-05 seconds ---
--- 1.8358230590820312e-05 seconds ---
--- 4.887580871582031e-05 seconds ---
--- 2.7179718017578125e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 6.723403930664062e-05 seconds ---
--- 1.6927719116210938e-05 seconds ---
--

--- 3.1948089599609375e-05 seconds ---
--- 3.695487976074219e-05 seconds ---
--- 3.719329833984375e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 1.5974044799804688e-05 seconds ---
--- 2.002716064453125e-05 seconds ---
--- 5.6743621826171875e-05 seconds ---
--- 1.5974044799804688e-05 seconds ---
--- 1.71661376953125e-05 seconds ---
--- 3.409385681152344e-05 seconds ---
--- 5.2928924560546875e-05 seconds ---
--- 6.29425048828125e-05 seconds ---
--- 3.1948089599609375e-05 seconds ---
--- 2.5987625122070312e-05 seconds ---
--- 2.4080276489257812e-05 seconds ---
--- 3.314018249511719e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 3.0040740966796875e-05 seconds ---
--- 3.790855407714844e-05 seconds ---
--- 3.910064697265625e-05 seconds ---
--- 4.1961669921875e-05 seconds ---
--- 3.0040740966796875e-05 seconds ---
--- 2.8848648071289062e-05 seconds ---
--- 1.8835067749023438e-05 seconds ---
--- 3.504753112792969e-05 seconds ---
--- 

In [51]:
len(links_2)

14343

In [52]:
with open('articles201806010621.pkl', 'wb') as f:
    pickle.dump(articles_2, f)


In [53]:
start_time = time.time()
dates_2=get_dates(links_2)
print("--- %s seconds ---" % (time.time() - start_time))

--- 3072.979101896286 seconds ---


In [54]:
start_time = time.time()
keywords_2=get_keywords(links_2)
print("--- %s seconds ---" % (time.time() - start_time))

--- 2903.334233045578 seconds ---


In [55]:
len(dates_2)

14343

In [56]:
len(keywords_2)

14343

In [57]:
len(articles_2)

14343

In [58]:
len(links_2)

14343

In [59]:
len(articles)

14452

In [60]:
len(links)

14452

In [61]:
len(dates)

14452

In [62]:
len(keywords)

14452

In [63]:
with open('dates201806010621.pkl', 'wb') as f:
    pickle.dump(dates_2, f)
    
with open('links201806010621.pkl', 'wb') as f:
    pickle.dump(links_2, f)
    
with open('keywords201806010621.pkl', 'wb') as f:
    pickle.dump(keywords_2, f)




In [64]:
df_1 = pd.DataFrame(np.column_stack([dates, links, keywords,articles]),columns=['date', 'link', 'keywords','article']) 

In [68]:
df_1['article'][0]


'Philadelphia Eagles tackle Lane Johnson hasn’t been shy about criticizing the New England Patriots, and he elaborated on what spurred many of those comments on a Steve Austin podcast published Tuesday. Jan 31, 2018; Bloomington, MN, USA; Philadelphia Eagles offensive tackle Lane Johnson during a press conference in advance of Super Bowl LII against the New England Patriots at Mall of America. Mandatory Credit: Jerry Lai-USA TODAY Sports“Here’s what pissed me off,” Johnson said when asked about his previous coments. “The Patriots, obviously, I respect their coach, I respect Bill , I respect Tom Brady. But just because the way that they won the Super Bowls, the Patriot Way, is that how everybody else is supposed to do the same thing? No, it’s not. And that’s what I got mad at, the arrogance by them. “There was obviously some stuff behind closed doors. Their owner (Robert Kraft) talking s—- to our owner (Jeffrey Lurie). Bill talking s—- to our head coach (Doug Pederson) before the game. 

In [69]:
df_2 = pd.DataFrame(np.column_stack([dates_2, links_2, keywords_2,articles_2]),columns=['date', 'link', 'keywords','article']) 

In [74]:
master_df=df_1.append(df_2)

In [75]:
df_2['date'].head()

0    June 1, 2018 /  3:05 AM
1    June 1, 2018 /  3:31 AM
2    June 1, 2018 /  3:05 AM
3    June 1, 2018 /  2:51 AM
4    June 1, 2018 /  2:42 AM
Name: date, dtype: object

In [76]:
df_1['date'].tail()

14447    May 31, 2018 /  3:33 AM
14448    May 31, 2018 /  4:23 AM
14449    May 31, 2018 /  4:07 AM
14450    May 31, 2018 /  4:02 AM
14451    May 31, 2018 /  3:33 AM
Name: date, dtype: object

In [79]:
os.getcwd()

'/Users/berkayadlim/Desktop'

In [80]:
master_df.to_csv("20180516-20180621.csv")

In [81]:
with open('master_df.pkl', 'wb') as f:
    pickle.dump(master_df, f)
